In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [2]:
df = pd.read_csv("processed_features.csv")
df.head()

,url_length,num_slashes,has_query,is_static,is_api,is_post,is_error,hour,is_night,requests_per_ip,unique_urls_per_ip,error_rate_per_ip,avg_size_per_ip,bot_ratio_per_ip,size
0,166,2,0,0,1,0,0,3,1,6,6,0.000000,31859.500000,1.0,30577
1,33,4,0,0,0,0,0,3,1,78,72,0.000000,10989.923077,0.0,5667
2,33,4,0,0,0,0,0,3,1,78,72,0.000000,10989.923077,0.0,5379
3,33,4,0,0,0,0,0,3,1,14,14,0.000000,10484.785714,1.0,1696
4,146,4,0,0,0,0,0,3,1,3299,3144,0.001819,41566.940588,0.0,41483


In [3]:
if 'ip' in df.columns:
    ip_addresses = df['ip']
    X = df.drop(columns=['ip'])
else:
    X = df.copy()

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
model = IsolationForest(
    n_estimators=100,
    contamination=0.05,
    random_state=42
)

model.fit(X_scaled)

IsolationForest(contamination=0.05, random_state=42)

In [7]:
anomaly_scores = model.decision_function(X_scaled)
anomaly_labels = model.predict(X_scaled)

In [8]:
results = X.copy()

results['anomaly_score'] = anomaly_scores
results['anomaly_label'] = anomaly_labels

if 'ip' in df.columns:
    results['ip'] = ip_addresses

In [9]:
results.to_csv("anomaly_scores.csv", index=False)

In [10]:
import joblib

joblib.dump(model, "isolation_forest.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']